In [1]:
from pyzotero import zotero
import os
import sys
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm.notebook import tqdm
tqdm.pandas()

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from code_utils.zotero import get_data_from_zotero
from code_utils.glutton import get_doi_glutton
from code_utils.utils import get_doi_cleaned,aplatir,wg_chap_to_dict,get_year_ipbes,check_doi_glutton
from code_utils.pickle import load_cache,write_cache
from code_utils.enriching_data_OpenAlex import get_countries_concepts_sdg,get_open_alex_data,get_open_alex_data_not_in_references

In [2]:
cached_openalex_data_ipbes = {}
cached_openalex_data_not_ipbes = {}

In [3]:
try:
    cached_openalex_data_ipbes = load_cache(cached_openalex_data_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_ipbes.pkl')
    cached_openalex_data_not_ipbes = load_cache(cached_openalex_data_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_not_ipbes.pkl')
except:
    #write_cache(cached_openalex_data_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_ipbes.pkl')
    #write_cache(cached_openalex_data_not_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_not_ipbes.pkl')
    print('oups')

5690 data in cached openalex data
58 data in cached openalex data


Read IPBES references in zotero files

In [4]:
ZOTERO_KEY = os.getenv('ZOTERO_KEY')
IPBES_ZOTERO_ID = '2333077'
COLLECTION_IDS = ['8DQ8YFJI','JR5LKU4U','LBFNF62B','7IMYH9U3','DRZLUY9K','KTPTCAVF','UWNSUNXQ','BJGWRJAK','JT8FJXP6']
CHAPTER_IDS=['1','2.1','2.2','2.3','3','4','5','6','glossary']
ALL_COLLECTIONS={}

In [5]:
ALL_COLLECTIONS=get_data_from_zotero(ZOTERO_KEY,IPBES_ZOTERO_ID,COLLECTION_IDS,CHAPTER_IDS,ALL_COLLECTIONS)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.zotero.org:443
DEBUG:urllib3.connectionpool:https://api.zotero.org:443 "GET /groups/2333077/collections/8DQ8YFJI/items?start=0&limit=100&format=json HTTP/1.1" 200 44992
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.zotero.org:443
DEBUG:urllib3.connectionpool:https://api.zotero.org:443 "GET /groups/2333077/collections/8DQ8YFJI/items?start=100&limit=100&format=json HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.zotero.org:443
DEBUG:urllib3.connectionpool:https://api.zotero.org:443 "GET /groups/2333077/collections/8DQ8YFJI/items?start=200&limit=100&format=json HTTP/1.1" 200 3987
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.zotero.org:443
DEBUG:urllib3.connectionpool:https://api.zotero.org:443 "GET /groups/2333077/collections/8DQ8YFJI/items?start=300&limit=100&format=json HTTP/1.1" 200 2
DEBUG:urllib3.connectionpool:Starting new HTTPS conn

In [6]:
df_ipbes_chap={}
for i in range(len(COLLECTION_IDS)):
    df_ipbes_chap[CHAPTER_IDS[i]]=pd.DataFrame(list(pd.DataFrame(ALL_COLLECTIONS[f'collection_chapter_{CHAPTER_IDS[i]}']).data))[['DOI','title','date','creators','tags']]
    df_ipbes_chap[CHAPTER_IDS[i]]['chap']=f'chap_{CHAPTER_IDS[i]}'
    print(f'{len(df_ipbes_chap[CHAPTER_IDS[i]][~pd.isna(df_ipbes_chap[CHAPTER_IDS[i]].DOI)])/len(df_ipbes_chap[CHAPTER_IDS[i]])*100} % doi available for chapter {CHAPTER_IDS[i]}')

48.03921568627451 % doi available for chapter 1
56.74373795761078 % doi available for chapter 2.1
58.08285946385053 % doi available for chapter 2.2
57.14285714285714 % doi available for chapter 2.3
59.76761074800291 % doi available for chapter 3
57.71358328211432 % doi available for chapter 4
53.789954337899545 % doi available for chapter 5
61.85125735687534 % doi available for chapter 6
31.53153153153153 % doi available for chapter glossary


Clean and enrich data 

In [8]:
df_ipbes=pd.concat(list(df_ipbes_chap.values())).reset_index()
del df_ipbes['index']

In [9]:
len(df_ipbes[~pd.isna(df_ipbes.DOI)])/len(df_ipbes)*100

57.41125613101671

In [10]:
df_ipbes.DOI.drop_duplicates()

0                                                    NaN
49       10.1890/1051-0761(2000)010[1251:ROTEKA]2.0.CO;2
51                       10.1016/j.gloenvcha.2013.12.012
52                               10.5751/ES-07868-200344
53                                                      
                              ...                       
12010                         10.1016/j.tree.2017.06.014
12013                                    10.2307/2388682
12016                                    10.2307/3060213
12017                        10.1016/BS.APAR.2016.02.009
12018                         10.2993/0278-0771-34.3.294
Name: DOI, Length: 5693, dtype: object

In [11]:
get_doi_glutton(df_ipbes.iloc[1,:])

In [ ]:
df_ipbes.loc[pd.isna(df_ipbes.DOI),'DOI']=df_ipbes.loc[pd.isna(df_ipbes.DOI),:].progress_apply(get_doi_glutton, axis=1)

  0%|          | 0/5123 [00:00<?, ?it/s]

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cloud.science-miner.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cloud.science-miner.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cloud.science-miner.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cloud.science-miner.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cloud.science-miner.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cloud.science-miner.com:443


In [ ]:
len(df_ipbes[~pd.isna(df_ipbes.DOI)])/len(df_ipbes)*100

77.36948241093435

In [ ]:
df_ipbes.loc[:,'DOI']=df_ipbes.loc[:,'DOI'].apply(lambda x: get_doi_cleaned(x))
df_ipbes.loc[:,'DOI']=df_ipbes.loc[:,'DOI'].apply(lambda x: None if str(x)[:4]=='http' else x)
df_ipbes['freq']=1
df_ipbes=df_ipbes.rename(columns={'DOI':'doi'})

In [29]:
df_ipbes.to_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes.json', orient='records')

In [ ]:
df_ipbes.columns

Enriching data through OpenAlex API integration

In [30]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes.json', orient='records')

In [31]:
df_unique_doi=df_ipbes.groupby(by=['doi'], dropna=True).agg({'chap': lambda x: list(x), 'freq': 'sum'})
df_unique_doi=df_unique_doi.reset_index()

In [32]:
#get data_OpenAlex from the API 
df_unique_doi.apply(lambda row: get_open_alex_data(cached_openalex_data_ipbes,row['doi']), axis= 1)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443
DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=doi:10.1002/(sici)1099-0771(199612)9:4%3C231::aid-bdm240%3E3.0.co;2-l HTTP/1.1" 200 3433
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443
DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=doi:10.1002/16-0188.1 HTTP/1.1" 200 3664
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443
DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=doi:10.1002/2013ef000188 HTTP/1.1" 200 5208
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443
DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=doi:10.1002/2013gb004654 HTTP/1.1" 200 5255
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443
DEBUG:urllib3.connectionpool:https://api.openalex

0       None
1       None
2       None
3       None
4       None
        ... 
5685    None
5686    None
5687    None
5688    None
5689    None
Length: 5690, dtype: object

In [33]:
len(cached_openalex_data_ipbes)

5690

In [34]:
write_cache(cached_openalex_data_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_ipbes.pkl')

In [35]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes.json')

In [36]:
df_unique_doi=df_ipbes['doi'].drop_duplicates().dropna()
df_unique_doi=pd.DataFrame({'doi':list(df_unique_doi)})

In [37]:
countries_list = []
concepts_list = []
sdg_list = []
topics_list = []
is_OA_available_list=[]
title_list=[]
year_list=[]
names_list=[]
rors_list=[]
institutions_names_list=[]
locations_names_list=[]
locations_ids_list=[]

for i,row in df_unique_doi.iterrows():
    countries,concepts,sdg,year,topics,doi,bool_OA,title,name,rors,institutions_names,locations_names,locations_ids=get_countries_concepts_sdg(cached_openalex_data_ipbes,row,ipcc=True,i=0)

    countries_list.append(countries)
    concepts_list.append(concepts)
    sdg_list.append(sdg)
    topics_list.append(topics)  
    is_OA_available_list.append(bool_OA)
    title_list.append(title)
    year_list.append(year)
    names_list.append(name)
    rors_list.append(rors)
    institutions_names_list.append(institutions_names)
    locations_names_list.append(locations_names)
    locations_ids_list.append(locations_ids)


df_unique_doi['countries'] = countries_list
df_unique_doi['concepts'] = concepts_list
df_unique_doi['sdg'] = sdg_list
df_unique_doi['topics'] = topics_list
df_unique_doi['is_OA_available'] = is_OA_available_list
df_unique_doi['year_OA'] = year_list
df_unique_doi['authors_name'] = names_list
df_unique_doi['rors'] = rors_list
df_unique_doi['institutions_names'] = institutions_names_list
df_unique_doi['locations_names'] = locations_names_list
df_unique_doi['locations_id'] = locations_ids_list

In [38]:
df_unique_doi.is_OA_available.value_counts()

is_OA_available
True     5668
False      22
Name: count, dtype: int64

In [39]:
df_unique_doi=df_unique_doi[df_unique_doi.is_OA_available]

In [40]:
df_unique_doi=df_unique_doi.reset_index()
del df_unique_doi['index']

In [41]:
df_unique_doi.to_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_OA.json', orient='records')

Ipbes references data

In [42]:
df_unique_doi=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_OA.json', orient='records')

In [43]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes.json')

In [44]:
df_ipbes=pd.merge(df_ipbes[['doi', 'title', 'date', 'creators', 'tags']], df_unique_doi, on='doi', how='inner')

In [45]:
df_ipbes['year']=df_ipbes['date'].apply(lambda x: get_year_ipbes(x) if pd.isna(x)==False else None)

In [46]:
df_ipbes['check']=df_ipbes.apply(lambda row: check_doi_glutton(row), axis=1)
df_ipbes.check.value_counts()

check
True     6363
False      80
Name: count, dtype: int64

In [47]:
df_ipbes[df_ipbes.check==False]

,doi,title,date,creators,tags,countries,concepts,sdg,topics,is_OA_available,year_OA,authors_name,rors,institutions_names,locations_names,locations_id,year,check
98,10.1007/s11027-013-9479-6,Combining backcasting and exploratory scenario...,2015,"[{'creatorType': 'author', 'firstName': 'Mathi...","[{'tag': 'ipbes-global_chapter1'}, {'tag': 'ip...",[NL],"[Backcasting, Computer science, Exploratory re...",None,"[Water resources management and optimization, ...",True,2013,"[[M. van Vliet, [NL]], [Kasper Kok, [NL]]]","[[https://ror.org/04qw24q55, NL], [https://ror...","[[Wageningen University & Research, NL], [Wage...",Mitigation and Adaptation Strategies for Globa...,1381-2386,2015,False
99,10.1007/s11027-013-9479-6,Combining backcasting and exploratory scenario...,2015,"[{'creatorType': 'author', 'firstName': 'Mathi...","[{'tag': 'ipbes-global_chapter1'}, {'tag': 'ip...",[NL],"[Backcasting, Computer science, Exploratory re...",None,"[Water resources management and optimization, ...",True,2013,"[[M. van Vliet, [NL]], [Kasper Kok, [NL]]]","[[https://ror.org/04qw24q55, NL], [https://ror...","[[Wageningen University & Research, NL], [Wage...",Mitigation and Adaptation Strategies for Globa...,1381-2386,2015,False
421,10.1016/j.enpol.2010.03.012,Distributional effects of taxing transport fuel,2012,"[{'creatorType': 'author', 'firstName': 'Thoma...",[{'tag': 'ipbes-global_chapter2.1_drivers'}],[SE],"[Opposition (politics), Economics, Surprise, C...","[{'id': '13', 'name': 'Climate action'}]","[Energy, Environment, and Transportation Polic...",True,2010,"[[Thomas Sterner, [SE]]]","[[https://ror.org/01tm6cn81, SE]]","[[University of Gothenburg, SE]]",Energy Policy,0301-4215,2012,False
652,10.1073/pnas.1220362110,The material footprint of nations,2015-05-19,"[{'creatorType': 'author', 'firstName': 'Thoma...","[{'tag': 'ipbes-global_chapter2.1_drivers'}, {...","[NO, AU, US, JP]","[Decoupling (probability), Natural resource, G...","[{'id': '8', 'name': 'Decent work and economic...","[Environmental Impact and Sustainability, Glob...",True,2013,"[[Thomas Wiedmann, [AU]], [Heinz Schandl, [AU]...","[[https://ror.org/03qn8fb07, AU], [https://ror...",[[Commonwealth Scientific and Industrial Resea...,Proceedings of the National Academy of Sciences,0027-8424,2015,False
653,10.1073/pnas.1220362110,The material footprint of nations,2015-05-19,"[{'creatorType': 'author', 'firstName': 'Thoma...","[{'tag': 'ipbes-global_chapter2.1_drivers'}, {...","[NO, AU, US, JP]","[Decoupling (probability), Natural resource, G...","[{'id': '8', 'name': 'Decent work and economic...","[Environmental Impact and Sustainability, Glob...",True,2013,"[[Thomas Wiedmann, [AU]], [Heinz Schandl, [AU]...","[[https://ror.org/03qn8fb07, AU], [https://ror...",[[Commonwealth Scientific and Industrial Resea...,Proceedings of the National Academy of Sciences,0027-8424,2015,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6289,10.1111/cobi.13056,Dynamics in the global protected‐area estate s...,06/2019,"[{'creatorType': 'author', 'firstName': 'Edwar...",[{'tag': 'ipbes-global_chapter6'}],"[DK, CH, GB]","[Protected area, Realm, Environmental resource...","[{'id': '14', 'name': 'Life below water'}]","[Conservation, Biodiversity, and Resource Mana...",True,2017,"[[Edward Lewis, [GB]], [Brian MacSharry, [GB]]...","[[https://ror.org/04570b518, GB], [https://ror...","[[World Conservation Monitoring Centre, GB], [...",Conservation Biology,0888-8892,2019,False
6291,10.1111/faf.12044,Evaluating the relative conservation value of ...,03/2015,"[{'creatorType': 'author', 'firstName': 'Marij...",[{'tag': 'ipbes-global_chapter6'}],[GB],"[Marine protected area, Fishing, Biomass (ecol...","[{'id': '14', 'name': 'Life below water'}]","[Coral and Marine Ecosystems Studies, Marine a...",True,2013,"[[Marija Sciberras, [GB]], [Stuart R. Jenkins,...","[[https://ror.org/006jb1a24, GB], [https://ror...","[[Bangor University, GB], [Bangor University, ...",Fish and Fisheries,1467

In [48]:
df_ipbes=df_ipbes[(df_ipbes.check) & (pd.isna(df_ipbes['is_OA_available'])==False)]

In [49]:
del df_ipbes['check']

In [50]:
df_ipbes.loc[:,'countries']=df_ipbes.loc[:,'countries'].apply(lambda x: str(x).replace('nan','None').replace('[]','None').replace('[None]','None') if ((str(x)=='nan')|(str(x)=='[None]')|((str(x)=='[]'))) else x )
df_ipbes.loc[:,'countries']=df_ipbes.loc[:,'countries'].apply(lambda x: ['None'] if x=='None' else x)

In [51]:
df_ipbes.to_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipbes\\data_ipbes.jsonl', orient='records',lines=True)

Find the data for constructing the learning model ( IPBES related or not )

In [52]:
df_ipbes=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipbes\\data_ipbes.jsonl', lines= True).dropna(subset=['year'])

In [53]:
df_ipbes['year'] = pd.to_numeric(df_ipbes['year'], errors='coerce')
df_ipbes=df_ipbes.dropna(subset=['year','doi', 'title','topics','locations_names','locations_id']).drop_duplicates(subset=['doi'])
year=df_ipbes.drop_duplicates(subset='doi').dropna(subset=['doi'])['year'].dropna().sort_values().apply(lambda x: int(x))
year_counts = pd.Series(list(year)).value_counts().to_dict()

In [54]:
dois=list(df_ipbes.doi.dropna().drop_duplicates())

In [55]:
sum(list(year_counts.values()))

5531

In [56]:
year_counts_not_ipbes={}

In [57]:
df_ipbes.topics.value_counts()[:50]

topics
[Land Use and Ecosystem Services, Conservation, Biodiversity, and Resource Management, Economic and Environmental Valuation]              34
[Conservation, Biodiversity, and Resource Management, Land Use and Ecosystem Services, Economic and Environmental Valuation]              33
[Conservation, Biodiversity, and Resource Management, Economic and Environmental Valuation, Land Use and Ecosystem Services]              31
[Marine and fisheries research, Coral and Marine Ecosystems Studies, Marine Bivalve and Aquaculture Studies]                              30
[Conservation, Biodiversity, and Resource Management, Economic and Environmental Valuation, Forest Management and Policy]                 24
[Marine and fisheries research, Marine Bivalve and Aquaculture Studies, Coral and Marine Ecosystems Studies]                              22
[Economic and Environmental Valuation, Conservation, Biodiversity, and Resource Management, Land Use and Ecosystem Services]              20
[Speci

In [58]:
for year in list(year_counts.keys()):   
    cached_openalex_data_not_ipbes[year]=[]
    year_counts_not_ipbes[year]=0
    while year_counts_not_ipbes[year]<year_counts[year]:
        get_open_alex_data_not_in_references(dois,cached_openalex_data_not_ipbes,year_counts,year_counts_not_ipbes,year,ipcc="ipbes")
    cached_openalex_data_not_ipbes[year]=cached_openalex_data_not_ipbes[year][:year_counts[year]]

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443
DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=has_doi:true,concepts_count:%3E0,publication_year:2016&sample=200&per-page=200 HTTP/1.1" 200 301587
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443


plus que 624 publications pour completer l'année 2016


DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=has_doi:true,concepts_count:%3E0,publication_year:2016&sample=200&per-page=200 HTTP/1.1" 200 320141
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443


plus que 518 publications pour completer l'année 2016


DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=has_doi:true,concepts_count:%3E0,publication_year:2016&sample=200&per-page=200 HTTP/1.1" 200 299196
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443


...

plus que 1 publications pour completer l'année 1976


DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=has_doi:true,concepts_count:%3E0,publication_year:1954&sample=200&per-page=200 HTTP/1.1" 200 149025
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443


plus que 1 publications pour completer l'année 1954


DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=has_doi:true,concepts_count:%3E0,publication_year:1936&sample=200&per-page=200 HTTP/1.1" 200 115425


plus que 1 publications pour completer l'année 1936


In [59]:
sum(list(year_counts.values()))

5531

In [60]:
len(aplatir(list(cached_openalex_data_not_ipbes.values())))


5531

In [61]:
len(cached_openalex_data_not_ipbes)

58

In [62]:
write_cache(cached_openalex_data_not_ipbes,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_not_ipbes.pkl')

In [63]:
df_not_ipbes=pd.DataFrame()
countries_list = []
concepts_list = []
sdg_list = []
topics_list = []
is_OA_available_list=[]
title_list=[]
year_list=[]
names_list=[]
rors_list=[]
institutions_names_list=[]
locations_names_list=[]
locations_ids_list=[]
doi_list=[]

for k in range(len(aplatir(list(cached_openalex_data_not_ipbes.values())))):
    countries,concepts,sdg,year,topics,doi,bool_OA,title,name,rors,institutions_names,locations_names,locations_ids=get_countries_concepts_sdg(cached_openalex_data=aplatir(list(cached_openalex_data_not_ipbes.values())),ipcc=False,i=k)

    countries_list.append(countries)
    concepts_list.append(concepts)
    sdg_list.append(sdg)
    topics_list.append(topics)  
    is_OA_available_list.append(bool_OA)
    title_list.append(title)
    year_list.append(year)
    names_list.append(name)
    rors_list.append(rors)
    institutions_names_list.append(institutions_names)
    locations_names_list.append(locations_names)
    locations_ids_list.append(locations_ids)
    doi_list.append(doi)


df_not_ipbes['doi'] = doi_list
df_not_ipbes['title'] = title_list
df_not_ipbes['countries'] = countries_list
df_not_ipbes['concepts'] = concepts_list
df_not_ipbes['sdg'] = sdg_list
df_not_ipbes['topics'] = topics_list
df_not_ipbes['is_OA_available'] = is_OA_available_list
df_not_ipbes['year'] = year_list
df_not_ipbes['authors_name'] = names_list
df_not_ipbes['rors'] = rors_list
df_not_ipbes['institutions_names'] = institutions_names_list
df_not_ipbes['locations_names'] = locations_names_list
df_not_ipbes['locations_id'] = locations_ids_list

In [64]:
df_not_ipbes

,doi,title,countries,concepts,sdg,topics,is_OA_available,year,authors_name,rors,institutions_names,locations_names,locations_id
0,https://doi.org/10.12962/j25023659.v2i1.1234,Orientasi Pada Pra Plotting Peta Bersistem Koo...,[],[Physics],"[{'id': '11', 'name': 'Sustainable cities and ...",[Decision Support System Applications],True,2016,"[(Yuwono Yuwono, []), (Adi Kurniawan, [])]",[],[],Jurnal Geosaintek,2460-9072
1,https://doi.org/10.3892/or.2016.5066,Quantification of plasma exosome is a potentia...,[JP],"[Exosome, Microvesicles, CD63, Cancer research...","[{'id': '3', 'name': 'Good health and well-bei...","[Extracellular vesicles in disease, MicroRNA i...",True,2016,"[(Yasunori Matsumoto, [JP]), (Masayuki Kano, [...","[(https://ror.org/01hjzeq58, JP), (https://ror...","[(Chiba University, JP), (Chiba University, JP...",Oncology Reports,1021-335X
2,https://doi.org/10.1021/acs.jctc.6b00641,Correction to Oxygen Vacancies of Anatase 101:...,[],"[Anatase, Density functional theory, Sensitivi...",None,[Electron and X-Ray Spectroscopy Techniques],True,2016,"[(Mai‐Anh Ha, []), (Anastassia N. Alexandrova,...",[],[],Journal of Chemical Theory and Computation,1549-9618
3,https://doi.org/10.1016/j.dental.2016.08.005,Intrinsically antibacterial adhesive systems b...,[GB],"[Materials science, Adhesive, Eugenol, Derivat...","[{'id': '2', 'name': 'Zero hunger'}]","[Synthesis of β-Lactam Compounds, Phosphorus c...",True,2016,"[(A. Almaroof, [GB]), (Sadia Niazi, [GB]), (Lu...","[(https://ror.org/0220mzb33, GB), (https://ror...","[(King's College London, GB), (King's College ...",Dental Materials,0109-5641
4,https://doi.org/10.18273/revion.v29n2-2016001,Impacto de la fermentación y secado sobre el c...,[CO],"[Horticulture, Biology]",None,"[Food Chemistry and Fat Analysis, Natural Prod...",True,2016,"[(Andrea Pallares Pallares, [CO]), (Mauren Est...","[(https://ror.org/00xc1d948, CO), (https://ror...","[(Industrial University of Santander, CO), (In...",Revista ION,0120-100X
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5526,https://doi.org/10.1016/0021-8928(74)90095-1,Contact problem for a stamp with narrow rectan...,[RU],"[Isotropy, Limit (mathematics), Mathematics, B...",None,"[Elasticity and Wave Propagation, Structural m...",True,1974,"[(Н. М. Бородачев, [RU]), (L. A. Galin, [RU])]",[],[],Journal of Applied Mathematics and Mechanics,0021-8928
5527,https://doi.org/10.1002/chin.197517067,ChemInform Abstract: NUCLEAR QUADRUPOLE RESONA...,[],"[Citation, Nuclear quadrupole resonance, Chin,...",None,"[Radioactive element chemistry and processing,...",True,1975,"[(F. DELAY, []), (M. Geoffroy, []), (E. A. C. ...",[],[],Chemischer Informationsdienst,0009-2975
5528,https://doi.org/10.11619/africa1964.1976.63,"URANO Tatsuo, International Relations in Afric...",[],"[Relation (database), International relations,...","[{'id': '17', 'name': 'Partnerships for the go...",[Global Peace and Security Dynamics],True,1976,"[(Tohru Yano, [])]",[],[],Journal of African Studies,0065-4140
5529,https://doi.org/10.1159/000150766,EVIDENCE REGARDING THE MULTIPLE MUTATION THEOR...,[],"[Mechanism (biology), Mutation, Genetics, Canc...","[{'id': '3', 'name': 'Good health and well-bei...",[Mathematical Biology Tumor Growth],True,1954,"[(C.O. Nordling, [])]",[],[],Human Heredity,0001-5652


In [65]:
df_not_ipbes.to_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipbes\\data_not_ipbes.jsonl', orient='records', lines= True)